In [ ]:
%matplotlib notebook

In [76]:
# Import dependencies

import numpy as np
import pandas as pd
import json
import requests
import matplotlib.pyplot as plt
from pprint import pprint
from config import api_keys


In [3]:
# Import data

filename1 = 'Resources/country_population.csv'
filename2 = 'Resources/fertility_rate.csv'
filename3 = 'Resources/life_expectancy.csv'

country_population = pd.read_csv(filename1)
fertility_rate = pd.read_csv(filename2)
life_expectancy = pd.read_csv(filename3)



In [32]:
# Cleaning up life expectancy data frame.
# Transposes dataframe and drops rows that aren't needed for calculations.


life_expectancy_transposed = life_expectancy.transpose()
life_expectancy_transposed.columns = life_expectancy_transposed.iloc[0]
life_expectancy_transposed = life_expectancy_transposed.drop(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'])
life_expectancy_transposed = life_expectancy_transposed.dropna(axis = 'columns',how = 'any')

life_expectancy_transposed.head()


Country Name,Aruba,Afghanistan,Angola,Albania,Arab World,United Arab Emirates,Argentina,Armenia,Antigua and Barbuda,Australia,...,"Venezuela, RB",Virgin Islands (U.S.),Vietnam,Vanuatu,World,Samoa,"Yemen, Rep.",South Africa,Zambia,Zimbabwe
1960,65.662,32.292,33.251,62.279,46.8251,52.265,65.022,65.971,62.115,70.8171,...,59.253,66.2249,59.041,46.441,52.5717,49.748,34.361,52.215,45.124,51.565
1961,66.074,32.742,33.573,63.298,47.4077,53.312,65.141,66.402,62.555,70.9732,...,59.829,66.488,59.722,47.044,53.0775,50.249,34.471,52.559,45.502,51.938
1962,66.444,33.185,33.914,64.187,47.9889,54.349,65.233,66.837,62.979,70.9424,...,60.395,66.7411,60.391,47.648,53.5016,50.75,34.739,52.894,45.869,52.3
1963,66.787,33.624,34.272,64.911,48.5722,55.375,65.311,67.276,63.385,70.9117,...,60.948,66.9962,61.024,48.251,54.0355,51.252,35.187,53.229,46.225,52.651
1964,67.113,34.06,34.645,65.461,49.1592,56.385,65.388,67.715,63.774,70.881,...,61.488,67.2604,61.585,48.853,54.7152,51.756,35.815,53.571,46.574,52.993


In [33]:
# Life expectancy averages from last 10 years

LE_10years = life_expectancy_transposed.iloc[range(46,57),:]

avg_life_10_years = pd.DataFrame(LE_10years.mean().sort_values(ascending = False))
avg_life_10_years = avg_life_10_years.rename(columns = {0 : 'Avg Life Expectancy (2006-2016)'})
avg_life_10_years = avg_life_10_years.reset_index()

avg_life_10_years.head()

,Country Name,Avg Life Expectancy (2006-2016)
0,"Hong Kong SAR, China",83.277384
1,Japan,83.052417
2,"Macao SAR, China",82.877000
3,Switzerland,82.427051
4,Italy,82.106430


In [54]:
# Set criteria for highest where highest is 80 years and above.
# 26 countries fall under this criteria

top_avg_life_10_years = avg_life_10_years.loc[avg_life_10_years["Avg Life Expectancy (2006-2016)"] > 80]
top_avg_life_10_years = top_avg_life_10_years.drop(index = 14).reset_index(drop = True)  # drops 'euro area' because it's not a country

top_avg_life_10_years

,Country Name,Avg Life Expectancy (2006-2016)
0,"Hong Kong SAR, China",83.277384
1,Japan,83.052417
2,"Macao SAR, China",82.877000
3,Switzerland,82.427051
4,Italy,82.106430
5,Iceland,82.091353
6,Spain,82.076940
7,Australia,81.841685
8,France,81.794235
9,Singapore,81.652328


In [55]:
# Set criteria for lowest life expectancy where lowest is 60 years and below.
# 30 countries fall under this criteria

bot_avg_life_10_years = avg_life_10_years.loc[avg_life_10_years["Avg Life Expectancy (2006-2016)"] < 60]
bot_avg_life_10_years = bot_avg_life_10_years.drop(index = [200,201,203,209,210,211,214]).reset_index(drop = True) # drops regions that aren't countries

bot_avg_life_10_years

,Country Name,Avg Life Expectancy (2006-2016)
0,"Gambia, The",59.842727
1,Liberia,59.817545
2,Benin,59.503273
3,Namibia,59.358091
4,Angola,58.501182
5,Malawi,58.037091
6,Togo,57.890455
7,Burkina Faso,57.529091
8,South Africa,57.387000
9,Uganda,57.375182


In [27]:
# Average life expectancy from 1960-2016

# avg_life = pd.DataFrame(life_expectancy_transposed.mean().sort_values(ascending = False))
# avg_life = avg_life.rename(columns = {0 : 'Avg Life Expectancy (1960-2016)'})
# avg_life = avg_life.reset_index()

# avg_life.head()


# Placeholder for this cell. Supplementary information. Potentially not needed

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

